In [1]:
import requests
import csv
import pandas as pd
import numpy as mp
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import time

options = Options()
options.add_argument("--disable-features=PreloadMediaEngagementData, MediaEngagementBypassAutoplayPolicies")
options.add_argument("--incognito --window-size=1000,800")
options.add_argument("--enable-popup-blocking")
options.add_extension("AutoplayStopper.crx")


In [ ]:
fileName = "MISS" #This will be for later, in case we want to save
wantedLink = "FGA" #This is the category we want
NUMLINKS = 5 #This is the number of links will be used,
NUMVIDSPERLINK = 25 #Number of video per each link

wantedFilterOn = True #Wanted Filter Bool, set to False if wanted list is empty.
#The title must have ALL substring in wanted list and NONE substring in unwanted list to get included.
wantedTitleStrList = ["MISS"] #List of substring to look for in the title.
unWantedTitleStrList = ["BLOCK"] #List of substring that will ignore in the title.


###########################################################################################################
linksClassDf = pd.DataFrame()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 5)

categoryDf = pd.read_csv('CategoryLinks.csv')
# Putting the wanted category in the wanted string
wantedDf = categoryDf[categoryDf['links'].str.contains(wantedLink)==True]

linkUsed = 0
for index, row in wantedDf.iterrows():
    main_url = "https://www.nba.com/" + row["links"]
    driver.get(main_url)
    #This is to set the page to list all the video
    try:
        closePopUp = wait.until(EC.presence_of_element_located((By.XPATH, "//button[@data-click='close']")))
        closePopUp.click()
    except:
        try:
            closePopUp = wait.until(EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Close Message']")))
            closePopUp.click()
        except:
            pass

    all = None
    while all is None:
        try:
            # connect
            all = wait.until(EC.presence_of_element_located((By.XPATH, "//option[@value = '-1']")))
        except:
             pass

    all.click()
    #driver.execute_script("arguments[0].click();", all)


    wait.until(EC.presence_of_element_located((By.XPATH,"//tr[@data-has-video='true']")))
    AllVidElement = driver.find_elements(By.XPATH, '//tr[@data-has-video="true"]')
    print(len(AllVidElement))
    vidDown = 0
    for vidButton in AllVidElement:
        driver.execute_script("arguments[0].click();", vidButton)
        videoElement = wait.until(EC.presence_of_element_located((By.XPATH, "//video")))
        vidLinks = videoElement.get_attribute("src")
        titleElement = driver.find_element(By.TAG_NAME, 'h2')
        titleString = titleElement.text
        unwantedBool = any(ele in titleString for ele in unWantedTitleStrList)
        wantedBool = (not wantedFilterOn or any(ele in titleString for ele in wantedTitleStrList))
        if not wantedBool or unwantedBool:
            continue

        #add it to our dataframe
        linksClassDf = linksClassDf.append([{'Highlight' : vidLinks, 'Title' : titleString}], ignore_index = True)
        vidDown = vidDown + 1
        if vidDown >= NUMVIDSPERLINK:
            break
    if linkUsed >= NUMLINKS:
        break

driver.close()


7010


C:\Users\danng\AppData\Local\Temp\ipykernel_45336\2893847049.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linksClassDf = linksClassDf.append([{'Highlight' : vidLinks, 'Title' : titleString}], ignore_index = True)
C:\Users\danng\AppData\Local\Temp\ipykernel_45336\2893847049.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linksClassDf = linksClassDf.append([{'Highlight' : vidLinks, 'Title' : titleString}], ignore_index = True)
C:\Users\danng\AppData\Local\Temp\ipykernel_45336\2893847049.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linksClassDf = linksClassDf.append([{'Highlight' : vidLinks, 'Title' : titleString}], ignore_index = True)
C:\Users\danng\AppData\Local\Temp\ipykernel_45336\2893847049.py:60: Fu

6666


C:\Users\danng\AppData\Local\Temp\ipykernel_45336\2893847049.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linksClassDf = linksClassDf.append([{'Highlight' : vidLinks, 'Title' : titleString}], ignore_index = True)
C:\Users\danng\AppData\Local\Temp\ipykernel_45336\2893847049.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linksClassDf = linksClassDf.append([{'Highlight' : vidLinks, 'Title' : titleString}], ignore_index = True)
C:\Users\danng\AppData\Local\Temp\ipykernel_45336\2893847049.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linksClassDf = linksClassDf.append([{'Highlight' : vidLinks, 'Title' : titleString}], ignore_index = True)
C:\Users\danng\AppData\Local\Temp\ipykernel_45336\2893847049.py:60: Fu

The code below will save the links we got to a csv file

In [ ]:
linksClassDf.head(100)
fileName = fileName + ".csv"
linksClassDf.to_csv(fileName)
